In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the trained model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("./saved_model_transformer_T2")
tokenizer = AutoTokenizer.from_pretrained("./saved_model_transformer_T2")

# Đảm bảo rằng mô hình và dữ liệu được tải lên GPU (nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm extract_information sử dụng mô hình đã huấn luyện
def extract_information(sentence):
    # Tokenize sentence
    tokenized_inputs = tokenizer(
        [sentence],  # Input là một câu
        is_split_into_words=False,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    # Tensor for device
    tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

    # predict label
    with torch.no_grad():
        outputs = model(**tokenized_inputs)

    predictions = outputs.logits.argmax(axis=2)

    # ID to label
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][0])
    pred_labels = [id_to_label[p.item()] for p in predictions[0]]

    # Filter data
    extracted_info = []
    for token, label in zip(tokens, pred_labels):
        if label != "O":  # Bỏ qua các token không có nhãn
            extracted_info.append((token, label))

    result = {}
    for token, label in extracted_info:
        entity_type = label.split("-")[-1]  # Bỏ tiền tố B- hoặc I-
        if entity_type not in result:
            result[entity_type] = []
        result[entity_type].append(token)

    return result

# Hàm xử lý các câu để in ra kết quả
def process_sentences(sentences):
    for idx, sentence in enumerate(sentences, 1):
        print(f"Sentence {idx}: {sentence}")
        result = extract_information(sentence)
        output = []
        for entity_type, entities in result.items():
            output.append(f"<{entity_type}> {' '.join(entities)}")
        print(" ".join(output))
        print("-" * 50)



In [3]:
label_list = ['O', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [4]:
test_sentences = [
    "Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa Thành phố Hồ Chí Minh.",
    "Hà Nội là thủ đô của Việt Nam.",
    "Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.",
    "Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.",
    "Facebook đã mở văn phòng mới tại Singapore.",
    "Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.",
    "Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.",
    "Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.",
    "Trời hôm nay đẹp và có nắng.",
    "Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợp tác với Đại học Quốc gia Hà Nội."
]

process_sentences(test_sentences)

Sentence 1: Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa Thành phố Hồ Chí Minh.
<PER> Nguyễn Văn An <ORG> trường Đại học B ##ách Khoa Thành phố Hồ Chí Minh
--------------------------------------------------
Sentence 2: Hà Nội là thủ đô của Việt Nam.
<LOC> Hà Nội Việt Nam
--------------------------------------------------
Sentence 3: Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.
<ORG> Công ty F ##P ##T Technology <LOC> Việt Nam
--------------------------------------------------
Sentence 4: Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.
<LOC> Hà Nội
--------------------------------------------------
Sentence 5: Facebook đã mở văn phòng mới tại Singapore.
<LOC> Singapore
--------------------------------------------------
Sentence 6: Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.
<PER> Nguyễn Thị Mai Châu <ORG> Tech ##com ##bank
--------------------------------------------------
Sentence 7: Thái Lan và Việt Nam sẽ thi đấu t

In [5]:
import spacy

# Giả sử mô hình đã được huấn luyện và lưu trữ tại path_to_save_model
nlp = spacy.load("IE Model")

# Hàm format kết quả thành định dạng <label> entity
def format_entities(doc):
    if not doc.ents:
        return "No entities found"  # Nếu không có thực thể nào, trả về thông báo
    formatted_output = []
    for ent in doc.ents:
        formatted_output.append(f"<{ent.label_}> {ent.text}")
    return " ".join(formatted_output)

# Danh sách các câu cần kiểm tra
test_sentences = [
    "Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa.",
    "Hà Nội là thủ đô của Việt Nam.",
    "Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.",
    "Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.",
    "Facebook đã mở văn phòng mới tại Singapore.",
    "Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.",
    "Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.",
    "Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.",
    "Trời hôm nay đẹp và có nắng.",
    "Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợp tác với Đại học Quốc gia Hà Nội."
]

# Xử lý từng câu trong danh sách test_sentences
for test_text in test_sentences:
    doc = nlp(test_text)
    formatted_output = format_entities(doc)
    print(f"Input: {test_text}\nOutput: {formatted_output}\n")


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'vi_pipeline' (0.0.0) was trained with spaCy v3.7.5 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Input: Ông Nguyễn Văn An là một giáo viên tại trường Đại học Bách Khoa.
Output: <B-PER> Nguyễn Văn An <B-ORG> trường <I-ORG> Đại học <I-ORG> Bách Khoa

Input: Hà Nội là thủ đô của Việt Nam.
Output: <B-LOC> Hà Nội <B-LOC> Việt Nam

Input: Công ty FPT Technology là một trong những công ty công nghệ lớn nhất ở Việt Nam.
Output: <B-ORG> Công ty <I-ORG> FPT <B-LOC> Việt Nam

Input: Sự kiện Techfest 2024 sẽ diễn ra tại Hà Nội.
Output: <B-LOC> Hà Nội

Input: Facebook đã mở văn phòng mới tại Singapore.
Output: <B-LOC> Singapore

Input: Bà Nguyễn Thị Mai Châu là giám đốc của ngân hàng Techcombank.
Output: <B-PER> Nguyễn Thị Mai Châu

Input: Thái Lan và Việt Nam sẽ thi đấu tại sân vận động Mỹ Đình.
Output: <B-LOC> Thái Lan <B-LOC> Việt Nam

Input: Ông Nguyễn Văn Bé vừa được trao giải thưởng sáng tạo của năm.
Output: <B-PER> Nguyễn Văn Bé

Input: Trời hôm nay đẹp và có nắng.
Output: No entities found

Input: Vào năm 2025, Google sẽ mở thêm một văn phòng tại thành phố Hồ Chí Minh và có kế hoạch hợ